In [2]:
from pyspark.sql import SparkSession

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("kiwisolver==1.2.0")
sc.install_pypi_package("seaborn")
sc.install_pypi_package("tabulate")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1604241517055_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/bf/4c/cb7da76f3a5e077e545f9cf8575b8f488a4e8ad60490838f89c5cdd5bb57/pandas-1.1.4-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/31/b9/6202dcae729998a0ade30e80ac00f616542ef445b088ec970d407dfd41c0/kiwisolver-1.2.0-cp37-cp37m-manylinux1_x86_64.whl

  Using cached https://files.pythonhosted.org/packages/bc/45/5118a05b0d61173e6eb12bc5804f0fbb6f196adb0a20e0b16efc2b8e98be/seaborn-0.11.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fa/cf/94686c3e2b21cba82904a2bbb014f7529d483021802a0116c3a256b00563/scipy-1.5.3-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/87/a6/8d7d06f6b69236a3c1818157875ceb1259ba0d9df4194f4fe138ffdc0f41/matplotlib-3.3.2-cp37-cp37m-manylinux1_x86_64.whl


In [3]:
import numpy as np
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import Row
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
in_bucket = "s3://tfmbigdata/Files_Segunda_Entrega/tickets*.json"
tickets = spark.read.format("json").load(in_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
tickets = tickets.distinct()
tickets_gr = tickets.groupby(['customerid','storeid','cardtype','datekey','ticketid','productid']).agg(
    F.sum('extendedamount').alias('extendedamount'),
    F.sum('originalamount').alias('originalamount'),
    F.sum('totaldiscount').alias('totaldiscount'),
    F.sum('quantity').alias('quantity')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
in_bucket = "s3://tfmbigdata/Files_Segunda_Entrega/products.json"
products = spark.read.format("json").load(in_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
in_bucket = "s3://tfmbigdata/Files_Segunda_Entrega/customers.json"
customers = spark.read.format("json").load(in_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
in_bucket = "s3://tfmbigdata/Files_Segunda_Entrega/stores.json"
stores = spark.read.format("json").load(in_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Coincidencia clientes

In [11]:
tickets_customerid = tickets.select('customerid').distinct()
customer_match = customers.join(tickets_customerid, customers.customerid == tickets_customerid.customerid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
print(tickets_customerid.count())
print(customers.count())
print(customer_match.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

49729
49998
3841

In [19]:
tickets.groupby().agg(F.countDistinct('ticketid')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT ticketid)|
+------------------------+
|                  922071|
+------------------------+

In [13]:
## Número de tickets de clientes coincidentes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# tickets_by_customer = tickets.groupby('customerid').count()
tickets.join(customers, tickets.customerid == customers.customerid).groupby().agg(F.countDistinct('ticketid')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT ticketid)|
+------------------------+
|                   77400|
+------------------------+

# Coincidencia con productos

In [24]:
tickets_productid = tickets.select('productid').distinct()
product_match = products.join(tickets_productid, products.productid == tickets_productid.productid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
print(tickets_productid.count())
print(products.count())
print(product_match.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6561
7917
2160

In [28]:
tickets.groupby().agg(F.count('productid')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|count(productid)|
+----------------+
|        13121119|
+----------------+

In [32]:
tickets.join(products, ['productid']).agg(F.count('productid')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|count(productid)|
+----------------+
|         4556223|
+----------------+